In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib
! pip install nltk

In [3]:
import pandas as pd

In [4]:
dataset = "Internship Tracker - ML_Dataset_email_classifier.csv"
dataset_kavya = "ML_Dataset_email_classifier_kavya.csv"
dataset_hamza = "ML_Dataset_email_classifier_Hamza.csv"
# Load the dataset
data_ary = pd.read_csv(dataset)
data_kavya = pd.read_csv(dataset_kavya)
data_hamza = pd.read_csv(dataset_hamza)

test_data = pd.concat([data_ary, data_kavya], ignore_index=True)
test_data = pd.concat([test_data, data_hamza], ignore_index=True)
data = data_ary.copy()

In [5]:
# choose a random subset of 100 rows from the data_kavya dataset and append it to the data dataset
data_kavya_subset = data_kavya.sample(n=len(data_kavya))
data = pd.concat([data, data_kavya_subset], ignore_index=True)
data = test_data

# Check the first few rows of your dataset
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 3 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            792 non-null    object 
 1   email body                               791 non-null    object 
 2   binary classification (expected output)  180 non-null    float64
dtypes: float64(1), object(2)
memory usage: 18.7+ KB


,binary classification (expected output)
count,180.000000
mean,0.383333
std,0.487555
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [6]:
# Check for missing values
data.isnull().sum()

# Drop rows with missing values (if necessary)
data.dropna(subset=['email subject', 'email body'], inplace=True)

#drop rows that are duplicates
data.drop_duplicates(subset=['email subject', 'email body'], inplace=True)

# Optionally, inspect cleaned data
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 767 entries, 0 to 791
Data columns (total 3 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            767 non-null    object 
 1   email body                               767 non-null    object 
 2   binary classification (expected output)  175 non-null    float64
dtypes: float64(1), object(2)
memory usage: 24.0+ KB


,binary classification (expected output)
count,175.000000
mean,0.394286
std,0.490099
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Define a function to clean the text
def clean_text(text):
    # Tokenize and lower case the text
    text = text.lower()
    tokens = word_tokenize(text)
    
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    
    return " ".join(cleaned_tokens)

# Apply the cleaning function to both 'email subject' and 'email body'
data['cleaned email subject'] = data['email subject'].apply(clean_text)
data['cleaned email body'] = data['email body'].apply(clean_text)


In [8]:
import os
def save_accuracy_score(accuracy, file_path='accuracy_scores.csv'):
    # Check if the file exists and is not empty
    if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
        # Create a new DataFrame with headers if the file does not exist or is empty
        scores_df = pd.DataFrame(columns=['Iteration', 'Accuracy'])
    else:
        # Load existing scores if the file exists and is not empty
        scores_df = pd.read_csv(file_path)

    # Append the new accuracy score
    new_score = {'Iteration': len(scores_df) + 1, 'Accuracy': accuracy}
    scores_df = pd.concat([scores_df, pd.DataFrame([new_score])], ignore_index=True)

    # Save the updated scores to the file
    scores_df.to_csv(file_path, index=False)

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [10]:
# Combine the subject and body into a single text feature
data['combined_text'] = data['cleaned email subject'] + " " + data['cleaned email body']

# Define your features (X) and labels (y)
X = data['combined_text']
y = data['binary classification (expected output)']  # This should be a column with binary labels (1 for job applications, 0 otherwise)

In [11]:
# Encode labels into binary format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [12]:
from sklearn.linear_model import LogisticRegression

# Separate labeled and unlabeled data
labeled_data = data[data['binary classification (expected output)'].notnull()]
before_labeled_data = len(labeled_data)

unlabeled_data = data[data['binary classification (expected output)'].isnull()]
before_unlabeled_data = len(unlabeled_data)

print("Number of labeled data points:", before_labeled_data)
print("Number of unlabeled data points:", before_unlabeled_data)
# check whether unlabeled data is empty
if(unlabeled_data.empty):
    print("Unlabeled data is empty")

Number of labeled data points: 175
Number of unlabeled data points: 592


In [34]:
def correct_prediction(incorrect_index, run_function):
    if incorrect_index is None or not run_function:
        return
    try:
        # Correct the prediction for the given index
        unlabeled_data.at[incorrect_index, 'predictions'] = 1
        print("This works")
    except:
        print("Index out of range")
        return

In [14]:
print(unlabeled_data)

                                         email subject  \
180  🚪 Open House Tomorrow at The Mark Atlanta – Fr...   
181                                Application Started   
182                                Application Started   
183                      Thanks for applying to Google   
184  Following up on your recent application to Google   
..                                                 ...   
787  Update regarding your application for 2024 Eng...   
788                    Thank You for Your Application!   
789                            Thank you for applying!   
790                           Thank  you for applying!   
791     We received your application. See what's next.   

                                            email body  \
180  You're Invited! Join us tomorrow, 01/17, from ...   
181  Hello! It’s been a while since we’ve heard fro...   
182  Hello! The Mark Atlanta is now at 50% capacity...   
183  Hi Kavya Adusumilli, Thanks for applying to Go...   
184  Thanks f

In [35]:
import joblib

tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
classifier = joblib.load('email_classifier.pkl')

predictions = classifier.predict(tfidf_vectorizer.transform(unlabeled_data['combined_text']))
print(len(predictions))
print(predictions)
unlabeled_data['predictions'] = predictions

592
[0 0 0 0 1 0 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 1 0 1 1 1 0 0 1 1
 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1
 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1
 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 0
 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 1
 1 1 1 0 1 0 1 0 1 0 

/var/folders/kq/cs03j9553ng9l_hc6_yh2qyw0000gn/T/ipykernel_9816/314634889.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlabeled_data['predictions'] = predictions


In [38]:
incorrect_index_list = [183, 213, 239, 375, 378, 386, 387, 389, 391, 393, 
                   394, 395, 405, 422, 429, 430, 431, 435, 453, 458, 
                   464, 466, 467, 470, 483, 488, 490, 494, 498, 500,
                   501, 503, 504, 507, 522, 523, 526, 527, 529, 533,
                   534, 546, 548, 553, 554, 561, 566, 567, 575, 576,
                   582, 583, 589, 603, 606, 607, 614, 621, 622, 631, 
                   632, 636, 639, 641, 642, 650, 653, 660, 664, 665,
                   666, 667, 668, 675, 686, 688, 690, 697, 698, 699,
                   703, 706, 714, 720, 725, 729, 732, 737, 739, 748,
                   753, 762, 764, 765, 768, 770, 771, 772, 776, 780,
                   781, 790, 791]
for i in incorrect_index_list:
    correct_prediction(i, True)

unlabeled_data.to_csv("new_sample_set_with_predictions.csv", index=False)

# print(unlabeled_data['combined_text'], unlabeled_data['predictions']) 
# print(unlabeled_data.head())
i = 0
for text, prediction in zip(unlabeled_data['email body'], unlabeled_data['predictions']):
    print(f"Index: {unlabeled_data.index[i]} Text: {text}\nPrediction: {prediction}\n")
    i+=1

This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works
This works

In [16]:
def mapPredictionsToIndices():
    index_prediction_map = {}
    for i, pred in enumerate(unlabeled_data['predictions']):
        if pred not in index_prediction_map:
            index_prediction_map[pred] = []
        index_prediction_map[pred].append(unlabeled_data.index[i])
    
    return index_prediction_map

In [39]:
index_prediction_map = mapPredictionsToIndices()

for prediction, indices in index_prediction_map.items():
    print(f"Prediction: {prediction}, Indices: {indices}")


Prediction: 0, Indices: [180, 181, 182, 185, 186, 188, 189, 190, 192, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 212, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 241, 242, 253, 254, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 360, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 381, 404, 413, 425, 440, 444, 445, 448, 449, 465, 476, 479, 618, 619, 620, 623, 628, 657, 662, 663, 677, 684, 702, 763, 779]
Prediction: 1, Indices: [183, 184, 187, 193, 194, 

In [40]:
# Train on labeled data
X_labeled = labeled_data['combined_text']
y_labeled = labeled_data['binary classification (expected output)']

tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))
X_labeled_tfidf = tfidf_vectorizer.fit_transform(X_labeled)

classifier = LogisticRegression()
classifier.fit(X_labeled_tfidf, y_labeled)

LogisticRegression()

In [41]:
# Predict probabilities on unlabeled data
X_unlabeled = unlabeled_data['combined_text']
try:
    print("Unlabeled data is not empty")
    X_unlabeled_tfidf = tfidf_vectorizer.transform(X_unlabeled)
except:
    print("Unlabeled data is empty")

Unlabeled data is not empty


In [42]:
probabilities = classifier.predict_proba(X_unlabeled_tfidf)
uncertainty = 1 - probabilities.max(axis=1)  # Higher uncertainty = lower confidence
unlabeled_data['uncertainty'] = uncertainty

# Sort by uncertainty
most_uncertain = unlabeled_data.sort_values(by='uncertainty', ascending=False)
uncertain_num = len(unlabeled_data)
most_uncertain_subset = most_uncertain.index[:uncertain_num]
most_uncertain[['email subject', 'email body', 'uncertainty']].index[:uncertain_num]
# print the email body of the uncertain_num most uncertain emails
for i in range(uncertain_num):
    print(most_uncertain.index[i], most_uncertain['uncertainty'].iloc[i], most_uncertain['email body'].iloc[i])


668 0.4998135560364242 Hi Syed Hamza Qadri , Thank you again for your interest in Illumina’s summer internship program! We wanted to let you know that we are still in the process of reviewing applications and hope to have a status update for you regarding 36781-JOB Bioinformatics Intern (Bachelors/Masters/PhD) (Evergreen) (Open) within the next few weeks. We have been fortunate to have received many applicants for our internship roles and it is important to us that we review every application and resume so thank you again for your patience with our process thus far. We hope you have a great day ahead! Sincerely, Illumina University Recruitment Team This email box is not monitored . Please do not reply to this email. Please do not reply to this message. This email address is not being monitored for responses. Please contact humanresources@illumina.com for questions or issues. This email was intended for hamzaqadri.careers@gmail.com Illumina, Inc. · www.illumina.com
667 0.499161569008462

/var/folders/kq/cs03j9553ng9l_hc6_yh2qyw0000gn/T/ipykernel_9816/1658425099.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlabeled_data['uncertainty'] = uncertainty


In [43]:
indices = []
indices = index_prediction_map[1]
# I want to label a set number of emails as job applications which ill do with the index
# of the most uncertain emails
# get the index of the uncertain_num most uncertain emails
# index = most_uncertain.index[:uncertain_num]
# print(index)
# label the emails as job applications
data.loc[indices, 'binary classification (expected output)'] = 1.0
# check if the emails have been labeled
print(data.loc[indices, 'binary classification (expected output)'])

183    1.0
184    1.0
187    1.0
193    1.0
194    1.0
      ... 
787    1.0
788    1.0
789    1.0
790    1.0
791    1.0
Name: binary classification (expected output), Length: 407, dtype: float64


In [44]:
import numpy as np
# label the other indices in the subset as not job applications
not_indices = [val for val in most_uncertain_subset if val not in indices]
data.loc[not_indices, 'binary classification (expected output)'] = 0.0
print(data.loc[not_indices, 'binary classification (expected output)'])

449    0.0
684    0.0
702    0.0
448    0.0
476    0.0
      ... 
270    0.0
280    0.0
235    0.0
262    0.0
185    0.0
Name: binary classification (expected output), Length: 185, dtype: float64


In [45]:
unlabeled_data = data[data['binary classification (expected output)'].isnull()]
after_unlabeled_data = len(unlabeled_data)
print(after_unlabeled_data)
labeled_data = data[data['binary classification (expected output)'].notnull()]
after_labeled_data = len(labeled_data)
print(after_labeled_data)

print(after_labeled_data - before_labeled_data)
print(after_unlabeled_data - before_unlabeled_data)

0
767
592
-592


In [47]:
# Define your features (X) and labels (y)
X = data['combined_text']
y = data['binary classification (expected output)']  # This should be a column with binary labels (1 for job applications, 0 otherwise)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))

# Fit and transform the training data, transform the test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train the classifier
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

save_accuracy_score(accuracy)


Accuracy: 0.9805194805194806
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97        56
           1       0.98      0.99      0.98        98

    accuracy                           0.98       154
   macro avg       0.98      0.98      0.98       154
weighted avg       0.98      0.98      0.98       154



In [49]:
import joblib

# Save the vectorizer and model
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(classifier, 'email_classifier.pkl')


['email_classifier.pkl']

In [50]:
# Load vectorizer and model
loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')
loaded_model = joblib.load('email_classifier.pkl')

# Preprocess and vectorize the new email
new_email = "This is a job application for the software engineer position."
new_email_tfidf = loaded_vectorizer.transform([new_email])

# Predict
prediction = loaded_model.predict(new_email_tfidf)
print("Prediction:", "Job Application" if prediction[0] == 1 else "Not a Job Application")


Prediction: Job Application
